In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from json import loads

In [2]:
url = 'https://www.youtube.com/@PW-Foundation/videos'

# Get the html by get method
r = requests.get(url)
r

<Response [200]>

In [3]:
# Create BeautifulSoup object
soup = BeautifulSoup(r.text, 'html.parser')
#soup.prettify()

In [4]:
all_script_tags = soup.findAll('script')

In [5]:
def script_tag_to_json(tags: list) -> dict:
    for tag in reversed(tags):
        text: str = tag.text
        if 'ytInitialData = {"responseContext"' in text:
            return loads(text[20:-1])

    raise ValueError('Required script tag not found in the given tags.')

In [6]:

data = script_tag_to_json(all_script_tags)

In [7]:
def get_contents_dict(data):
    return data['contents']['twoColumnBrowseResultsRenderer']['tabs'][1]['tabRenderer']['content']['richGridRenderer']['contents']

Q1. Write a python program to extract the video URL of the first five videos.

In [8]:
from unittest import result


def get_videoUrl(data:dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append('https://www.youtube.com/watch?v=' +
                      contents[i]['richItemRenderer']['content']['videoRenderer']['videoId'])

    return result

print(len(get_videoUrl(data)))
get_videoUrl(data)

5


['https://www.youtube.com/watch?v=pecNtI_QXEI',
 'https://www.youtube.com/watch?v=PXcgTSs-_p8',
 'https://www.youtube.com/watch?v=JGJv4jlf8aQ',
 'https://www.youtube.com/watch?v=0dMJLkO-SSw',
 'https://www.youtube.com/watch?v=0CW4VbfH4kk']

Q2. Write a python program to extract the URL of the video thumbnails of the first five videos.

In [9]:
def get_thumbnails(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['thumbnail']['thumbnails'][-1]['url'])

    return result

print(len(get_thumbnails(data)))
get_thumbnails(data)

5


['https://i.ytimg.com/vi/pecNtI_QXEI/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLA74lvwupTBs1HyW_kRpPPRR7m6oA',
 'https://i.ytimg.com/vi/PXcgTSs-_p8/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDO-RkTO4cXpwKl7fPqmIfBPL4epA',
 'https://i.ytimg.com/vi/JGJv4jlf8aQ/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLDqp4egkUL4WsQyMUk3y38CGQJflw',
 'https://i.ytimg.com/vi/0dMJLkO-SSw/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLABjl1NbhXu4_x-u0GM837jIU4niQ',
 'https://i.ytimg.com/vi/0CW4VbfH4kk/hqdefault.jpg?sqp=-oaymwEjCNACELwBSFryq4qpAxUIARUAAAAAGAElAADIQj0AgKJDeAE=&rs=AOn4CLCNdz05xox55dHNhjHfwG3mI1a3mg']

Q3. Write a python program to extract the title of the first five videos.


In [10]:
def get_title(data: dict, n:int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['title']['runs'][-1]['text'])

    return result

print(len(get_title(data)))
get_title(data)

5


['POWER SHARING in 40 mins || Class - 10th Boards',
 'DEVELOPMENT in 45 mins || Class - 10th Boards',
 'REAL NUMBERS in 60 mins || Class - 10th Boards',
 'RESOURCES AND DEVELOPMENT in 45 mins || Class - 10th Boards',
 'LIGHT REFLECTION AND REFRACTION in 35 mins || Class - 10th Boards']

Q4. Write a python program to extract the number of views of the first five videos.

In [11]:
def get_viwes(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(int(contents[i]['richItemRenderer']['content']['videoRenderer']['viewCountText']['simpleText']
                      [:-6].replace(',', '')))

    return result

print(len(get_viwes(data)))
get_viwes(data)

5


[5869, 5654, 8895, 11853, 25471]

Q5. Write a python program to extract the time of posting of video for the first five videos.

In [12]:
def get_time_of_posting(data: dict, n: int = 5):
    contents = get_contents_dict(data)

    if n > 30:
        raise ValueError('Max Limit is 30.')

    result = []
    for i in range(n):
        result.append(contents[i]['richItemRenderer']['content']['videoRenderer']['publishedTimeText']['simpleText'])

    return result

print(len(get_time_of_posting(data)))
get_time_of_posting(data)

5


['17 hours ago', '1 day ago', '2 days ago', '3 days ago', '4 days ago']

Note: Save all the data scraped in the above questions in a CSV file.

In [13]:
from turtle import title


def get_channel_video_details(data: dict, n: int):
    thumbnails = get_thumbnails(data, n)
    time_of_posting = get_time_of_posting(data, n)
    titles = get_title(data, n)
    video_urls = get_videoUrl(data, n)
    viwes=get_viwes(data, n)

    main_data = list(zip(titles, video_urls, thumbnails, viwes,time_of_posting))
    
    df = DataFrame.from_dict(main_data)
    df.rename(
        columns={
            0: 'Title',
            1: 'Video Url',
            2: 'Thumbnail',
            3: 'Views',
            4: 'Time of Posting'
        }, inplace=True)

    return df

In [14]:
channel_data = get_channel_video_details(data, 5)

In [15]:
channel_data.to_csv('PW-Foundation.csv', index=False)

In [16]:
import pandas as pd

pd.read_csv('PW-Foundation.csv', index_col=False)

,Title,Video Url,Thumbnail,Views,Time of Posting
0,POWER SHARING in 40 mins || Class - 10th Boards,https://www.youtube.com/watch?v=pecNtI_QXEI,https://i.ytimg.com/vi/pecNtI_QXEI/hqdefault.j...,5869,17 hours ago
1,DEVELOPMENT in 45 mins || Class - 10th Boards,https://www.youtube.com/watch?v=PXcgTSs-_p8,https://i.ytimg.com/vi/PXcgTSs-_p8/hqdefault.j...,5654,1 day ago
2,REAL NUMBERS in 60 mins || Class - 10th Boards,https://www.youtube.com/watch?v=JGJv4jlf8aQ,https://i.ytimg.com/vi/JGJv4jlf8aQ/hqdefault.j...,8895,2 days ago
3,RESOURCES AND DEVELOPMENT in 45 mins || Class ...,https://www.youtube.com/watch?v=0dMJLkO-SSw,https://i.ytimg.com/vi/0dMJLkO-SSw/hqdefault.j...,11853,3 days ago
4,LIGHT REFLECTION AND REFRACTION in 35 mins || ...,https://www.youtube.com/watch?v=0CW4VbfH4kk,https://i.ytimg.com/vi/0CW4VbfH4kk/hqdefault.j...,25471,4 days ago
